## Visualize path points

In [1]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from geolib import geohash as geolib

/Users/colombmo/Documents/Lavoro/SWICE/Software/tests/OSMnx/osmnx/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Defined legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 160px; height: 300px; 
                 background-color: white; border:2px solid grey; z-index:9999; 
                 font-size:14px;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; Walking &nbsp; <i class="fa fa-square" style="color:#7FC97F"></i><br>
     &nbsp; On Bicycle &nbsp; <i class="fa fa-square" style="color:#BDAED4"></i><br>
     &nbsp; Train &nbsp; <i class="fa fa-square" style="color:#FDBF85"></i><br>
     &nbsp; Bus &nbsp; <i class="fa fa-square" style="color:#FFFF99"></i><br>
     &nbsp; Electric Bus &nbsp; <i class="fa fa-square" style="color:#FFFF99"></i><br>
     &nbsp; Car &nbsp; <i class="fa fa-square" style="color:#386CB0"></i><br>
     &nbsp; Tram &nbsp; <i class="fa fa-square" style="color:#F0027F"></i><br>
     &nbsp; Plane &nbsp; <i class="fa fa-square" style="color:#BE5B17"></i><br>
     &nbsp; Boat &nbsp; <i class="fa fa-square" style="color:#BE5B17"></i><br>
      </div>
     '''

In [3]:
df = gpd.read_file('data/pathpoints.csv')

In [4]:
# Keep only relevant data
df = df[['time_range', 'day_of_week', 'month', 'mode_of_transport', 'geohash', 'direction']]
df[:5]

,time_range,day_of_week,month,mode_of_transport,geohash,direction
0,18-21,TUESDAY,8,BUS,u0m70dcm,EAST
1,18-21,TUESDAY,8,BUS,u0m70ddq,NORTH_WEST
2,18-21,TUESDAY,8,BUS,u0m70dfh,EAST
3,6-9,WEDNESDAY,8,ON_BICYCLE,u0m70fqv,NORTH_EAST
4,6-9,WEDNESDAY,8,ON_BICYCLE,u0m70fqk,EAST


In [5]:
## Get a coordinate from a geohash, adding a small random offset to avoid overlapping
def geohash_to_coordinate(geohash):
    lat, lon = geolib.decode(geohash)
    #lat = float(lat) + 0.00000001#(random.random() - 0.5) * 0.00000001
    #lon = float(lon) + 0.00000001#(random.random() - 0.5) * 0.00000001
    return [float(lat), float(lon)]

In [6]:
# Group by geohashes
df_grouped = df.groupby('geohash').size().reset_index(name='count')

In [7]:
## Extract from df a list of all start_geohashes and end_geohashes
geohashes = list(df['geohash'])
len(geohashes)
# Convert the list of geohashes to a dataframe
geohashes_df = gpd.GeoDataFrame(geohashes, columns=['geohash'])
# Decode geohashes to latitude and longitude
df_grouped['latitude'] = df_grouped['geohash'].apply(lambda x: geohash_to_coordinate(x)[0])
df_grouped['longitude'] = df_grouped['geohash'].apply(lambda x: geohash_to_coordinate(x)[1])
# Create a base map
m = folium.Map(location=[df_grouped['latitude'].mean(), df_grouped['longitude'].mean()], zoom_start=10)

# Prepare data for the heatmap
heat_data = [[row['latitude'], row['longitude'], row['count']] for index, row in df_grouped.iterrows()]

# Create a heatmap layer
HeatMap(heat_data).add_to(m)

# Display the map
m.save('maps/pathpoints_heatmap.html')
!open -a Arc maps/pathpoints_heatmap.html